In [1]:
import dataAck
import portfolio
allTickers = dataAck.getAllTickersPlain()
import params

In [ ]:
while True:
    import random
    tickerToTrade = allTickers[random.randint(0, len(allTickers)) - 1]
    print(tickerToTrade)
    
    tData = dataAck.getTrainingData(tickerToTrade)
    joinedData = None
    validTickers = None

    
    
    if tData is None:
        dataAck.logModel("Cache", {
            "type":"miss",
            "ticker":tickerToTrade,
            "day":str(portfolio.getToday())
        })

        tickersToPull = dataAck.getDataSourcesForTicker(tickerToTrade)
        print(tickersToPull)

        pulledData, validTickers = dataAck.downloadTickerData(tickersToPull)

        joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])
        
        dataAck.storeTrainingData(tickerToTrade, (joinedData, validTickers))
    else:
        joinedData = tData[0]
        validTickers = tData[1]
        dataAck.logModel("Cache", {
            "type":"hit",
            "ticker":tickerToTrade,
            "day":str(portfolio.getToday())
        })
        
    
    sManager = dataAck.seriesManager(validTickers)
    print(sManager.describe())

    import time
    import warnings
    import numpy as np
    warnings.filterwarnings("ignore")
    ##GET ALGOS INITIALLY GOOD
    runsSeen = 0
    while True:
        s = sManager.createSeries()
        while s.checkValidity(s.transformJoinedData(joinedData[:"2016-01-01"])) == False:
            s = sManager.createSeries()
            

        try:
            for defaultWindowSize in [5, 10, 22, 44]:
                for trees in [25, 50, 100, 150, 300]:
                    for predictionLength in [2, 3, 5, 7]:
                        if random.uniform(0,1) < 0.5: ##RANDOMLY SKIP
                            continue
                        b = dataAck.algoBlob(s, defaultWindowSize, trees, predictionLength, tickerToTrade)
                        algoReturn, factorReturn, predictions =  b.makePredictions(portfolio.prepareDataForModel(b, joinedData), daysToCheck = None, earlyStop = True)
                        if algoReturn is None:
                            toLog = factorReturn
                            if np.isnan(toLog["sharpe"]) == True:
                                raise ValueError('SHARPE IS NAN SO FAULTY SERIES')
                                
                            toLog["modelDescription"] = str(b.describe())
                            dataAck.logModel("Model Stopped Early", toLog)
                            print("Model Stopped Early", toLog)
                            continue
                        metrics = dataAck.vizResults(algoReturn[:-252], factorReturn[:-252], False)
                        print("TRAIN:", metrics)
                        if np.isnan(metrics["SHARPE"]) == True:
                            raise ValueError('SHARPE IS NAN SO FAULTY SERIES')
                        if metrics["SHARPE"] > 1.0 and metrics["ACTIVITY"] > 0.7 and metrics["BETA"] < 0.6:
                            ##STORE
                            testMetrics = dataAck.vizResults(algoReturn[-252:], factorReturn[-252:], False)
                            print("TEST:", testMetrics)
                            print("TODAY:", b.makeTodayPrediction(portfolio.prepareDataForModel(b, joinedData)))

                            dataAck.storeModel(b, metrics, testMetrics)
                        else:
                            dataAck.logModel("Model Skipped", {
                                "modelDescription":str(b.describe()),
                                "sharpe":metrics["SHARPE"], ##OVERLOADED IN FAIL
                                "activity":metrics["ACTIVITY"],
                                "beta":metrics["BETA"]
                            })
        except:
            print("FAILED", s.describe())
            dataAck.logModel("Series Failed", {
                "seriesDescription":str(s.describe())
            })

        runsSeen += 1

        if runsSeen > 10:
            ##START NEW TICKER
            dataAck.logModel("Search Update", {
                "message":"restarting search with different ticker",
                "currentTicker":tickerToTrade
            })
            break

        

EWW
ATTEMPTING PULL c250a7e24fa415a13153da12c270dd7016506ab8e506574cb9c22bc9
['AGG', 'DIA', 'DVY', 'DXJ', 'EFA', 'EWC', 'EWG', 'EWH', 'EWJ', 'EWT', 'EWU', 'EWY', 'FEZ', 'FXE', 'GDX', 'GLD', 'IAU', 'IBB', 'IEF', 'IJH', 'IJR', 'ITB', 'IVE', 'IVV', 'IVW', 'IWB', 'IWD', 'IWF', 'IWM', 'IWN', 'IWO', 'IWR', 'IYF', 'IYR', 'IYT', 'KBE', 'KRE', 'LQD', 'MDY', 'OEF', 'OIH', 'QQQ', 'RSP', 'SDY', 'SH', 'SHY', 'SLV', 'SMH', 'SOXX', 'SPY', 'TIP', 'TLT', 'USO', 'VB', 'VBR', 'VFH', 'VGK', 'VGT', 'VIG', 'VNQ', 'VO', 'VTI', 'VTV', 'VUG', 'XBI', 'XHB', 'XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY', 'XRT', 'EWW']
DXJ
AGG
DVY
EFA
DIA
EWH
EWJ
EWG
EWC
EWT
EWY
FXE
FEZ
EWU
GLD
IAU
GDX
IBB
IEF
ITB
IJH
IJR
IVE
IVV
IVW
IWB
IWD
IWM
IWF
IWR
IWO
IYT
IWN
IYF
IYR
KBE
KRE
LQD
SDY
OIH
RSP
OEF
QQQ
SH
MDY
SHY
SLV
USO
TIP
SOXX
SMH
VB
TLT
SPY
VBR
VGK
VGT
VIG
VFH
VNQ
VO
VTI
XBI
VUG
VTV
XHB
XLB
XLF
XLE
XLK
XLI
XLP
XLU
XLV
XLY
XRT
EWW
STORING c250a7e24fa415a13153da12c270dd7016506ab8e506574cb9c22bc9
['AGG', 'DIA',

Process ForkProcess-160:
Process ForkProcess-156:
Process ForkProcess-158:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/prohb125/walkforwardTrader/dataAck.py", line 31

FAILED ('XLF', 3, None, None, 2)
(5, ('XLY', 20, 7, None, 2), 3, 'EWW')
SERIES (5, ('XLY', 20, 7, None, 2), 3, 'EWW') WINDOW 5 TREES 25
FAILED ('XLY', 20, 7, None, 2)
(5, ('OIH', 6, None, None, 2), 5, 'EWW')
SERIES (5, ('OIH', 6, None, None, 2), 5, 'EWW') WINDOW 5 TREES 25
CONTINUING SHARPE: 0.553847516703 BETA: 0.293472031442
CONTINUING SHARPE: 1.3481300369 BETA: 0.330807692385
CONTINUING SHARPE: 0.661357223359 BETA: 0.313767486211
THREAD  0 PROGRESS: 0.18518518518518517
THREAD  1 PROGRESS: 0.18518518518518517
THREAD  2 PROGRESS: 0.18518518518518517
THREAD  3 PROGRESS: 0.18518518518518517
THREAD  4 PROGRESS: 0.18518518518518517
THREAD  5 PROGRESS: 0.18518518518518517
THREAD  6 PROGRESS: 0.18633540372670807
THREAD  9 PROGRESS: 0.18633540372670807
THREAD  7 PROGRESS: 0.18633540372670807
THREAD  8 PROGRESS: 0.18633540372670807
THREAD  12 PROGRESS: 0.18633540372670807
THREAD  0 PROGRESS: 0.37037037037037035
THREAD  11 PROGRESS: 0.18633540372670807
THREAD  14 PROGRESS: 0.18633540372670807


Process ForkProcess-348:
Process ForkProcess-355:
Process ForkProcess-346:
Traceback (most recent call last):
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/prohb125/walkforwardTrader/dataAck.py", line 310, in runDayChunking
    pred = self.runDay(xVals[:int(i)], yVals[:int(i)], xVals[int(i)+44])
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/prohb125/anaconda3/lib/python3

FAILED ('OIH', 6, None, None, 2)
(5, ('SH', 5, None, None, 1), 3, 'EWW')
SERIES (5, ('SH', 5, None, None, 1), 3, 'EWW') WINDOW 5 TREES 25
FAILED ('SH', 5, None, None, 1)
(5, ('ITB', 16, None, None, 2), 2, 'EWW')
SERIES (5, ('ITB', 16, None, None, 2), 2, 'EWW') WINDOW 5 TREES 25
FAILED ('ITB', 16, None, None, 2)
(5, ('LQD', 2, 6, 9, 3), 2, 'EWW')
SERIES (5, ('LQD', 2, 6, 9, 3), 2, 'EWW') WINDOW 5 TREES 25
Model Stopped Early {'sharpe': 1.4725362594466989, 'beta': 0.0083733835723732043, 'activity': 0.16666666666666666, 'period': 'first 252 days', 'modelDescription': "((5, ('LQD', 2, 6, 9, 3), 2, 'EWW'), 5, 25)"}
(5, ('LQD', 2, 6, 9, 3), 5, 'EWW')
SERIES (5, ('LQD', 2, 6, 9, 3), 5, 'EWW') WINDOW 5 TREES 25
FAILED ('LQD', 2, 6, 9, 3)


KeyboardInterrupt: 